# Notebook de entrega a Kaggle, implementación de modelo híbrido regresión lineal + XGBOOST II

Primero se va a implementar un modelo en el que no se tiene en cuenta el precio del petróleo, en función de los resultados, se tendrá en un segundo modelo. Asimismo, en un primer caso se implementará un dataset X de entrada al modelo de regresión con análisis de la tendencia y transformada de Fourier para captar temporalidad y en el modelo XGBOOST esto ya no se tendrá en cuenta. En función de los resultados, también se introducirá esto en el segundo modelo.

In [ ]:
# Setup notebook
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import warnings
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from joblib import Parallel, delayed
import category_encoders as ce
from matplotlib.offsetbox import AnchoredText
import math
from sklearn.preprocessing import LabelEncoder

Importación de los archivos

In [3]:
store_sales_train = pd.read_csv('train.csv')
store_sales_test = pd.read_csv('test.csv')
oil = pd.read_csv('oil.csv', parse_dates=['date'])
holidays_events = pd.read_csv('holidays_events.csv', dtype={'type': 'category','locale': 'category',
                            'locale_name': 'category','description': 'category','transferred': 'bool',}, parse_dates=['date'], infer_datetime_format=True)
transactions = pd.read_csv('transactions.csv', parse_dates=['date'])
stores = pd.read_csv('stores.csv')

En este caso, sí se tendrán en cuenta los datos del precio del petróleo pero sólo en el modelo XGBoost en con el que se trata de ajustar los residuos

In [4]:
oil['date'] = pd.to_datetime(oil['date'])
oil['date'] = oil.date.dt.to_period('D')
oil.set_index('date', inplace=True)
oil = oil.rename(columns={'dcoilwtico':'precio'})
#Se rellenan los nulos correspondientes al fin de semana con el valor del viernes
oil['precio'].ffill(inplace=True)

In [5]:
def make_lags_precio(ts, lags):
    return pd.concat(
        {
            f'y_lag_precio_{i}': ts.shift(i)
            for i in range(1, lags + 1)
        },
        axis=1)

In [6]:
lags_oil = make_lags_precio(oil['precio'],2)
oil = oil.join(lags_oil)
oil = oil.loc['2017-01-01':,:]

Se pasa la fecha a periodo para crear el dataset X  y se utilizarán únicamente los datos de 2017 para ajustar el modelo

In [7]:
store_sales_train['date'] = pd.to_datetime(store_sales_train['date'])
store_sales_train['date'] = store_sales_train.date.dt.to_period('D')
store_sales_test['date'] = pd.to_datetime(store_sales_test['date'])
store_sales_test['date'] = store_sales_test.date.dt.to_period('D')

store_sales_train_17 = store_sales_train.loc[store_sales_train['date'] > '2017-01-01'].set_index(['store_nbr', 'family', 'date']).sort_index()

In [8]:
y_train_LR = store_sales_train_17.unstack(['store_nbr', 'family'])
y_train_LR = y_train_LR['sales']

fourier = CalendarFourier(freq="M", order=4)
dp = DeterministicProcess(
    index=y_train_LR.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier],
    drop=True
)
X1_train_LR = dp.in_sample()

In [9]:
holidays_events = holidays_events.set_index('date').to_period('D')

holidays_17 = (
    holidays_events
    .query("locale in ['National']")
    .loc['2017':'2017-08-31', ['type','transferred']]
)

holidays_17.drop(holidays_17[holidays_17['transferred']==True].index,axis=0,inplace=True)
holidays_17.rename(columns = {'type':'event_type'}, inplace=True)

holidays_17.loc[holidays_17['event_type'].isin(['Holiday','Transfer','Additional', 'Bridge']),'Free_day']=1
holidays_17.loc[holidays_17['event_type'].isin(['Event']),'Event']=1
holidays_17['Free_day'].fillna(0,inplace=True)
holidays_17['Event'].fillna(0,inplace=True)

display(holidays_17)

,event_type,transferred,Free_day,Event
date,,,,
2017-01-02,Transfer,False,1.0,0.0
2017-02-27,Holiday,False,1.0,0.0
2017-02-28,Holiday,False,1.0,0.0
2017-04-14,Holiday,False,1.0,0.0
2017-05-01,Holiday,False,1.0,0.0
2017-05-13,Additional,False,1.0,0.0
2017-05-14,Event,False,0.0,1.0
2017-05-26,Transfer,False,1.0,0.0
2017-08-11,Transfer,False,1.0,0.0


In [10]:
X2_train_LR = X1_train_LR.join(holidays_17[['Free_day','Event']], on='date').fillna(0.0)

model_LR =Ridge(fit_intercept=False).fit(X2_train_LR, y_train_LR)
y_pred_LR = pd.DataFrame(model_LR.predict(X2_train_LR), index=y_train_LR.index, columns=y_train_LR.columns)

In [11]:
X_test_LR = dp.out_of_sample(steps=16)
X_test_LR.index.names=['date']
X_test_LR = X_test_LR.join(holidays_17[['Free_day','Event']], on='date').fillna(0.0)

y_fore_LR = pd.DataFrame(model_LR.predict(X_test_LR), index= X_test_LR.index, columns=y_train_LR.columns)

In [12]:
residuos_LR = y_train_LR-y_pred_LR
residuos_LR = pd.DataFrame(residuos_LR.stack(['store_nbr', 'family']))
residuos_LR.reset_index(inplace=True)

In [13]:
familias = list(store_sales_train.family.unique())

In [14]:
imp_lags = {'AUTOMOTIVE': [4,6,7],
'BABY CARE':[2],
'BEAUTY': [1,2,3],
'BEVERAGES':[7],
'BOOKS':[1,4,5],
'BREAD/BAKERY':[2,7],
'CLEANING':[1],
'DAIRY':[2,7],
'DELI':[2,7],
'EGGS': [2,7],
'FROZEN FOODS': [1],
'GROCERY I':[2],
'GROCERY II':[1,5],
'HARDWARE': [5,6],
'HOME AND KITCHEN I':[1,2],
'HOME AND KITCHEN II':[1,2,3],
'HOME APPLIANCES':[1,2,3,5,6],
'HOME CARE':[5,7],
'LADIESWEAR':[2,7],
'LAWN AND GARDEN':[1,2],
'LINGERIE':[1,2,5],
'LIQUOR,WINE,BEER':[1,2,6],
'MAGAZINES':[1,6],
'MEATS':[5],
'PLAYERS AND ELECTRONICS':[7],
'POULTRY':[5],
'PREPARED FOODS':[1,7],
'PRODUCE':[2],
'SCHOOL AND OFFICE SUPPLIES':[1,4,5,7],
'SEAFOOD':[1,3,5]}

In [15]:
promotions_train = pd.DataFrame(store_sales_train_17['onpromotion'])
promotions_train.reset_index(inplace=True)
promotions_test = pd.DataFrame(store_sales_test[['onpromotion','date','store_nbr','family']])

In [16]:
residuos_LR.rename(columns={0:'residuos'},inplace=True)
residuos_LR['date'] = residuos_LR['date'].dt.to_timestamp()
promotions_train['date'] = promotions_train['date'].dt.to_timestamp()
residuos_LR_train = residuos_LR.merge(promotions_train, on=['date' ,'store_nbr','family'],how='left').fillna(0.0)
residuos_LR_train = residuos_LR_train.merge(stores, on=['store_nbr'],how='left').fillna(0.0)

In [17]:
residuos_LR_test = promotions_test.merge(stores, on=['store_nbr'],how='left').fillna(0.0)

In [18]:
holidays_17_reg = (
    holidays_events
    .query("locale in ['Regional']")
    .loc['2017':'2017-08-31', ['type','locale_name','transferred']]
)

holidays_17_loc = (
    holidays_events
    .query("locale in ['Local']")
    .loc['2017':'2017-08-31', ['type','locale_name','transferred']]
)

holidays_17_reg.drop(holidays_17_reg[holidays_17_reg['transferred']==True].index,axis=0,inplace=True)
holidays_17_loc.drop(holidays_17_loc[holidays_17_loc['transferred']==True].index,axis=0,inplace=True)

holidays_17_reg.rename(columns = {'type':'event_type'}, inplace=True)
holidays_17_loc.rename(columns = {'type':'event_type'}, inplace=True)

holidays_17_reg.loc[holidays_17_reg['event_type'].isin(['Holiday','Transfer','Additional', 'Bridge']),'Free_day']=1
holidays_17_loc.loc[holidays_17_loc['event_type'].isin(['Holiday','Transfer','Additional', 'Bridge']),'Free_day']=1

#No hay eventos locales ni regionales
#holidays_17_reg.loc[holidays_17_reg['event_type'].isin(['Event']),'Event']=1
#holidays_17_loc.loc[holidays_17_loc['event_type'].isin(['Event']),'Event']=1

#holidays_17_reg['Event'].fillna(0,inplace=True)
#holidays_17_loc['Event'].fillna(0,inplace=True)

holidays_17_reg['Free_day'].fillna(0,inplace=True)
holidays_17_loc['Free_day'].fillna(0,inplace=True)

holidays_17_reg.drop(['event_type','transferred'],axis=1,inplace=True)
holidays_17_loc.drop(['event_type','transferred'],axis=1,inplace=True)

holidays_17_reg['locale_name']=holidays_17_reg['locale_name'].astype('object')
holidays_17_loc['locale_name']=holidays_17_loc['locale_name'].astype('object')

In [19]:
residuos_LR_train = residuos_LR_train.merge(holidays_17_reg, left_on=['date','state'], right_on=['date','locale_name'], how='left').fillna(0.0)
residuos_LR_train = residuos_LR_train.merge(holidays_17_loc, left_on=['date','city'], right_on=['date','locale_name'],how='left').fillna(0.0)

residuos_LR_test = residuos_LR_test.merge(holidays_17_reg, left_on=['date','state'], right_on=['date','locale_name'], how='left').fillna(0.0)
residuos_LR_test = residuos_LR_test.merge(holidays_17_loc, left_on=['date','city'], right_on=['date','locale_name'],how='left').fillna(0.0)


residuos_LR_train['Free_day'] = residuos_LR_train['Free_day_x'] + residuos_LR_train['Free_day_y']
residuos_LR_train.drop(['city','state','locale_name_x','Free_day_x','locale_name_y','Free_day_y'], axis=1, inplace=True)

residuos_LR_test['Free_day'] = residuos_LR_test['Free_day_x'] + residuos_LR_test['Free_day_y']
residuos_LR_test.drop(['city','state','locale_name_x','Free_day_x','locale_name_y','Free_day_y'], axis=1, inplace=True)

In [20]:
residuos_LR_test['residuos']=np.nan

In [21]:
ohencoder = ce.OneHotEncoder(cols='type', use_cat_names=True)

residuos_LR_train = ohencoder.fit_transform(residuos_LR_train)
residuos_LR_test = ohencoder.transform(residuos_LR_test)


residuos_LR_train['date'] = pd.to_datetime(residuos_LR_train['date'])
residuos_LR_train['date'] = residuos_LR_train.date.dt.to_period('D')

residuos_LR_train.set_index('date',inplace=True)
residuos_LR_train['dayofweek'] = residuos_LR_train.index.dayofweek
residuos_LR_train['dayofmonth'] = residuos_LR_train.index.day

residuos_LR_test.set_index('date',inplace=True)
residuos_LR_test['dayofweek'] = residuos_LR_test.index.dayofweek
residuos_LR_test['dayofmonth'] = residuos_LR_test.index.day

In [44]:
fourier = CalendarFourier(freq="M", order=4)
dp = DeterministicProcess(
    index = residuos_LR_train.index.unique(),
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier],
    drop=True
)

In [43]:
residuos_LR_train = residuos_LR_train.join(dp.in_sample())
residuos_LR_train

,store_nbr,family,residuos,onpromotion,type_D,type_B,type_C,type_E,type_A,cluster,...,"s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)"
date,,,,,,,,,,,,,,,,,,,,,
2017-01-02,1,AUTOMOTIVE,3.323478,0,1,0,0,0,0,13,...,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,0.724793,0.688967
2017-01-02,1,BABY CARE,0.000000,0,1,0,0,0,0,13,...,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,0.724793,0.688967
2017-01-02,1,BEAUTY,-2.366353,0,1,0,0,0,0,13,...,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,0.724793,0.688967
2017-01-02,1,BEVERAGES,20.523578,31,1,0,0,0,0,13,...,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,0.724793,0.688967
2017-01-02,1,BOOKS,-0.267723,0,1,0,0,0,0,13,...,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,0.724793,0.688967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-15,54,POULTRY,-12.166820,0,0,0,1,0,0,3,...,0.0,0.0,0.299363,-0.954139,-0.571268,0.820763,0.790776,-0.612106,-0.937752,0.347305
2017-08-15,54,PREPARED FOODS,3.989036,0,0,0,1,0,0,3,...,0.0,0.0,0.299363,-0.954139,-0.571268,0.820763,0.790776,-0.612106,-0.937752,0.347305
2017-08-15,54,PRODUCE,-24.258599,76,0,0,1,0,0,3,...,0.0,0.0,0.299363,-0.954139,-0.571268,0.820763,0.790776,-0.612106,-0.937752,0.347305


In [47]:
dp_test = dp.out_of_sample(steps=16)
residuos_LR_test = residuos_LR_test.join(dp_test)
residuos_LR_test

,onpromotion,store_nbr,family,type_D,type_B,type_C,type_E,type_A,cluster,Free_day,...,"s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)"
2017-08-16,0,1,AUTOMOTIVE,1,0,0,0,0,13,0.0,...,0.0,0.0,0.101168,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958
2017-08-16,0,1,BABY CARE,1,0,0,0,0,13,0.0,...,0.0,0.0,0.101168,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958
2017-08-16,2,1,BEAUTY,1,0,0,0,0,13,0.0,...,0.0,0.0,0.101168,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958
2017-08-16,20,1,BEVERAGES,1,0,0,0,0,13,0.0,...,0.0,0.0,0.101168,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958
2017-08-16,0,1,BOOKS,1,0,0,0,0,13,0.0,...,0.0,0.0,0.101168,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-31,1,9,POULTRY,0,1,0,0,0,6,0.0,...,0.0,0.0,-0.201299,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967
2017-08-31,0,9,PREPARED FOODS,0,1,0,0,0,6,0.0,...,0.0,0.0,-0.201299,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967
2017-08-31,1,9,PRODUCE,0,1,0,0,0,6,0.0,...,0.0,0.0,-0.201299,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967
2017-08-31,9,9,SCHOOL AND OFFICE SUPPLIES,0,1,0,0,0,6,0.0,...,0.0,0.0,-0.201299,0.979530,-0.394356,0.918958,-0.571268,0.820763,-0.724793,0.688967


In [48]:
residuos_LR = pd.concat([residuos_LR_train,residuos_LR_test])

Aquí se implementan los datos del petróleo

In [50]:
residuos_LR = residuos_LR.join(oil)
residuos_LR['precio'].ffill(inplace=True)
residuos_LR['y_lag_precio_1'].ffill(inplace=True)
residuos_LR['y_lag_precio_2'].ffill(inplace=True)

In [33]:
def make_lags_residuos(ts, lags):
    return pd.concat(
        {
            f'y_lag_residuos_{i}': ts.shift(i)
            for i in lags
        },
        axis=1)

def make_lags_promotions(ts, lags):
    return pd.concat(
        {
            f'y_lag_promotions_{i}': ts.shift(i)
            for i in range(1, lags + 1)
        },
        axis=1)

def make_leads_promotions(ts, leads):
    return pd.concat(
        {
            f'y_lead_promotions_{i}': ts.shift(-i)
            for i in range(1, leads + 1)
        },
        axis=1)

In [55]:
residuos_LR = residuos_LR.reset_index().rename(columns={'index':'date'})

In [57]:
residuos_dict = {}

for familia in familias:
    residuos_dict[familia] = pd.DataFrame(residuos_LR[residuos_LR['family']==familia])
    residuos_dict[familia]['tienda'] = residuos_dict[familia]['store_nbr']
    residuos_dict[familia] = residuos_dict[familia].drop('family', axis=1)
    residuos_dict[familia] = residuos_dict[familia].set_index(['store_nbr', 'date']).sort_index()

lags_promo_dict = {}
leads_promo_dict = {}
for familia in familias:
    lags_promo_dict[familia]=make_lags_promotions(residuos_dict[familia].onpromotion, lags = 3)
    leads_promo_dict[familia]=make_leads_promotions(residuos_dict[familia].onpromotion, leads = 3)
    residuos_dict[familia] = residuos_dict[familia].join(lags_promo_dict[familia])
    residuos_dict[familia] = residuos_dict[familia].join(leads_promo_dict[familia])

In [58]:
lags_resi_dict = {}
for familia in imp_lags.keys():
    lags_resi_dict[familia]=make_lags_residuos(residuos_dict[familia].residuos, lags = imp_lags[familia])
    residuos_dict[familia] = residuos_dict[familia].join(lags_resi_dict[familia])

Va a volver a dividirse en entrenamiento y test el dataset de residuos que se tiene para cada familia y al dataset de entrenamiento se le van a quitar las 8 primeras fechas, pues son las que podrían contener nans y por parte del set de validación, las leads de promociones se van a poner a 0 en las tres últimas fechas

In [60]:
store_sales_test['date'].unique()

<PeriodArray>
['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19', '2017-08-20',
 '2017-08-21', '2017-08-22', '2017-08-23', '2017-08-24', '2017-08-25',
 '2017-08-26', '2017-08-27', '2017-08-28', '2017-08-29', '2017-08-30',
 '2017-08-31']
Length: 16, dtype: period[D]

In [61]:
residuos_train_dict = {}
residuos_test_dict = {}
for familia in familias:
    residuos_dict[familia].reset_index(inplace=True)
    residuos_train_dict[familia] = residuos_dict[familia].loc[(residuos_dict[familia]['date']>'2017-01-08') & (residuos_dict[familia]['date']<'2017-08-16')]
    residuos_test_dict[familia] = residuos_dict[familia].loc[residuos_dict[familia]['date']>='2017-08-16']
    residuos_test_dict[familia].set_index('date',inplace=True)
    residuos_test_dict[familia].loc['2017-08-31','y_lead_promotions_1':'y_lead_promotions_3'] = 0
    residuos_test_dict[familia].loc['2017-08-30','y_lead_promotions_2':'y_lead_promotions_3'] = 0
    residuos_test_dict[familia].loc['2017-08-29','y_lead_promotions_3'] = 0
    residuos_train_dict[familia] = residuos_train_dict[familia].set_index(['store_nbr','date']).sort_index()
    residuos_test_dict[familia].reset_index(inplace=True)
    residuos_test_dict[familia] = residuos_test_dict[familia].set_index(['store_nbr','date']).sort_index()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [63]:
X_train_dict = {}
y_train_dict ={}

for familia in familias:
    X_train_dict[familia] = residuos_train_dict[familia].drop('residuos',axis=1)
    y_train_dict[familia] = pd.DataFrame(residuos_train_dict[familia]['residuos'])

In [64]:
dict_params = {"n_estimators": [100, 150],
               "max_depth": [5, 8, 11],
               "learning_rate": [0.06, 0.02, 0.1],
               "objective": ["reg:squarederror"],
               "tree_method": ["hist"],
               "subsample": [0.6,1]
              }

In [65]:
modelos_dict_XGB1 = {}
y_pred_dict_XGB1 = {}

for familia in familias:
    modelos_dict_XGB1[familia] = GridSearchCV(estimator=XGBRegressor(random_state=50),
                         param_grid=dict_params,
                         refit=True,
                         verbose=1,
                         n_jobs=-1,
                         return_train_score=True,
                         scoring="neg_mean_squared_error"
                         )

In [67]:
for familia in familias:
    modelos_dict_XGB1[familia].fit(X_train_dict[familia], y_train_dict[familia])
    print(f'Ajustado modelo de {familia} \n {modelos_dict_XGB1[familia].best_params_}')
    y_pred_dict_XGB1[familia] =pd.DataFrame(modelos_dict_XGB1[familia].predict(X_train_dict[familia]), index=y_train_dict[familia].index, columns=y_train_dict[familia].columns)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Ajustado modelo de AUTOMOTIVE 
 {'learning_rate': 0.02, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'subsample': 0.6, 'tree_method': 'hist'}
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Ajustado modelo de BABY CARE 
 {'learning_rate': 0.02, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'subsample': 0.6, 'tree_method': 'hist'}
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Ajustado modelo de BEAUTY 
 {'learning_rate': 0.02, 'max_depth': 5, 'n_estimators': 150, 'objective': 'reg:squarederror', 'subsample': 1, 'tree_method': 'hist'}
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Ajustado modelo de BEVERAGES 
 {'learning_rate': 0.02, 'max_depth': 11, 'n_estimators': 150, 'objective': 'reg:squarederror', 'subsample': 0.6, 'tree_method': 'hist'}
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Ajustado modelo de BOOKS 
 {'

In [89]:
for familia in familias:
    print(f'El error cuadrático medio de {familia} en train es {mean_squared_error(y_train_dict[familia], y_pred_dict_XGB1[familia])}')

El error cuadrático medio de AUTOMOTIVE en train es 12.076126303035407
El error cuadrático medio de BABY CARE en train es 0.2901244307453971
El error cuadrático medio de BEAUTY en train es 7.345192442264462
El error cuadrático medio de BEVERAGES en train es 142093.04773457217
El error cuadrático medio de BOOKS en train es 0.16792637556343842
El error cuadrático medio de BREAD/BAKERY en train es 3747.8989294931334
El error cuadrático medio de CELEBRATION en train es 68.51833104431383
El error cuadrático medio de CLEANING en train es 98904.53846057015
El error cuadrático medio de DAIRY en train es 2415.761334971013
El error cuadrático medio de DELI en train es 1370.2545373445942
El error cuadrático medio de EGGS en train es 1477.377288409466
El error cuadrático medio de FROZEN FOODS en train es 859.4305191383146
El error cuadrático medio de GROCERY I en train es 208801.10640142203
El error cuadrático medio de GROCERY II en train es 124.872103455988
El error cuadrático medio de HARDWARE e

Bucle para ir actualizando los valores de test

In [76]:
X1_test_dict = {}
resul_actual = {}

for familia in familias:
    X1_test_dict[familia] = residuos_test_dict[familia].loc[:,'onpromotion':'y_lead_promotions_3']
    X1_test_dict[familia] = X1_test_dict[familia].reset_index()
    resul_actual[familia] = pd.concat([y_train_dict[familia].reset_index(),X1_test_dict[familia].reset_index()[['date','store_nbr']]]).set_index(['store_nbr','date']).sort_index()

In [77]:
fechas = pd.date_range(start="2017-08-16", end = "2017-08-31").to_period(freq='D').tolist()
lags_resi_test={}
X2_iteracion = {}
X1_iteracion = {}
X_iteracion = {}
pred_iter = {}

for fecha in fechas:
    for familia in familias:
        X1_iteracion[familia] = X1_test_dict[familia][X1_test_dict[familia]['date']==fecha]
        X1_iteracion[familia] = X1_iteracion[familia].set_index(['store_nbr','date']).sort_index()
        if familia in imp_lags.keys():
            lags_resi_test[familia] = make_lags_residuos(resul_actual[familia].residuos, lags = imp_lags[familia])
            lags_resi_test[familia].reset_index(inplace=True)
            X2_iteracion[familia] = lags_resi_test[familia][lags_resi_test[familia]['date']==fecha]
            X2_iteracion[familia] = X2_iteracion[familia].set_index(['store_nbr','date']).sort_index()
            X_iteracion[familia] = X1_iteracion[familia].join(X2_iteracion[familia])
        else:
            X_iteracion[familia] = X1_iteracion[familia]
        pred_iter[familia] = pd.DataFrame(modelos_dict_XGB1[familia].predict(X_iteracion[familia]), index=X_iteracion[familia].index, columns = y_train_dict[familia].columns)
        resul_actual[familia] = resul_actual[familia].reset_index()
        resul_actual[familia] = resul_actual[familia][resul_actual[familia]['date'] != fecha]
        resul_actual[familia] = resul_actual[familia].set_index(['store_nbr', 'date']).sort_index()
        resul_actual[familia] = pd.concat([resul_actual[familia],pred_iter[familia]]).sort_index()    

In [82]:
y_fore_dict = {}

for familia in familias:
    y_fore_dict[familia] = resul_actual[familia].reset_index()
    y_fore_dict[familia] = y_fore_dict[familia][y_fore_dict[familia]['date']>='2017-08-16'].set_index(['store_nbr','date']).sort_index()

In [83]:
residuos_ajustados_test = pd.DataFrame()
for familia in familias:
    y_fore_dict[familia]['family']=familia
    residuos_ajustados_test = pd.concat([residuos_ajustados_test, y_fore_dict[familia]])

Se procede a sumar las predicciones iniciales y estos valores de los residuos para obtener las predicciones finales de ventas

In [84]:
residuos_ajustados_test = residuos_ajustados_test.reset_index().set_index(['store_nbr', 'family', 'date']).sort_index().rename(columns={0:'predicted_residuos'})
y_fore_LR = y_fore_LR.stack(['store_nbr','family']).reset_index().set_index(['store_nbr', 'family', 'date']).sort_index().rename(columns={0:'predicted_sales'})
y_test_pred = pd.DataFrame()
y_test_pred['adjusted_predicted_sales'] = y_fore_LR['predicted_sales'] + residuos_ajustados_test['residuos']
y_test_pred.loc[y_test_pred['adjusted_predicted_sales']<0,'adjusted_predicted_sales'] = 0
y_test_pred

adjusted_predicted_sales
store_nbr family     date                                
1         AUTOMOTIVE 2017-08-16                  4.429315
                     2017-08-17                  4.145696
                     2017-08-18                  5.249636
                     2017-08-19                  4.250701
                     2017-08-20                  1.865893
...                                                   ...
54        SEAFOOD    2017-08-27                  3.295022
                     2017-08-28                  3.493285
                     2017-08-29                  3.736024
                     2017-08-30                  4.746768
                     2017-08-31                  3.550331

[28512 rows x 1 columns]

In [85]:
store_sales_test = store_sales_test.set_index(['store_nbr', 'family', 'date']).sort_index()

In [87]:
y_submit = y_test_pred.join(store_sales_test.id)
y_submit.columns = ['sales','id']
y_submit = y_submit.set_index('id')
y_submit.sort_index(inplace=True)
y_submit

,sales
id,
3000888,4.429315
3000889,0.070345
3000890,6.181251
3000891,2644.795083
3000892,0.426809
...,...
3029395,371.849522
3029396,111.582354
3029397,1256.048934


In [88]:
y_submit.to_csv('submission7.csv')

**El RMSLE en este caso es de 0.54989, por lo que mejora respecto al Hybrid sin Deterministic Process, pero no mejora al caso en que no se consideran los residuos**

Quitando las familiar para las que peor salen los residuos

In [96]:
residuos_familias_bien = residuos_ajustados_test.reset_index()
familias_malas=familias.copy()
for familia in ['BABY CARE', 'BEVERAGES', 'BOOKS', 'BREAD/BAKERY', 'CELEBRATION', 'DAIRY', 'EGGS', 'FROZEN FOODS', 'GROCERY I', 'HOME AND KITCHEN I', 'LAWN AND GARDEN', 'PLAYERS AND ELECTRONICS', 'POULTRY', 'PREPARED FOODS', 'PRODUCE', 'SEAFOOD']:
    familias_malas.remove(familia)
residuos_familias_bien.loc[residuos_familias_bien['family'].isin(familias_malas),'residuos']=0

In [97]:
residuos_familias_bien = residuos_familias_bien.set_index(['store_nbr', 'family', 'date']).sort_index()

In [98]:
y_test_pred_familias_bien = pd.DataFrame()
y_test_pred_familias_bien['adjusted_predicted_sales'] = y_fore_LR['predicted_sales'] + residuos_familias_bien['residuos']
y_test_pred_familias_bien.loc[y_test_pred_familias_bien['adjusted_predicted_sales']<0,'adjusted_predicted_sales'] = 0
y_test_pred_familias_bien

adjusted_predicted_sales
store_nbr family     date                                
1         AUTOMOTIVE 2017-08-16                  4.293369
                     2017-08-17                  4.191120
                     2017-08-18                  5.475544
                     2017-08-19                  4.404853
                     2017-08-20                  1.765618
...                                                   ...
54        SEAFOOD    2017-08-27                  3.295022
                     2017-08-28                  3.493285
                     2017-08-29                  3.736024
                     2017-08-30                  4.746768
                     2017-08-31                  3.550331

[28512 rows x 1 columns]

In [100]:
y_submit2 = y_test_pred_familias_bien.join(store_sales_test.id)
y_submit2.columns = ['sales','id']
y_submit2 = y_submit2.set_index('id')
y_submit2.sort_index(inplace=True)
y_submit2

,sales
id,
3000888,4.293369
3000889,0.070345
3000890,3.532349
3000891,2644.795083
3000892,0.426809
...,...
3029395,371.849522
3029396,111.582354
3029397,1256.048934


In [101]:
y_submit.to_csv('submission8.csv')

# Rangom Forest

In [105]:
dict_params = {"n_estimators": [100, 180, 250],
               "max_depth": [5, 8, 11],
               "criterion": ["mse"],
               "max_features": [None],
               "bootstrap": [True],
               "oob_score": [True],
               "max_samples": [0.6, 1]
              }

In [106]:
modelos_dict_RF1 = {}
y_pred_dict_RF1 = {}

for familia in familias:
    modelos_dict_RF1[familia] = GridSearchCV(estimator=RandomForestRegressor(random_state=50),
                         param_grid=dict_params,
                         refit=True,
                         verbose=1,
                         n_jobs=-1,
                         return_train_score=True,
                         scoring="neg_mean_squared_error"
                         )

In [108]:
for familia in familias:
    modelos_dict_RF1[familia].fit(X_train_dict[familia], y_train_dict[familia])
    print(f'Ajustado modelo de {familia} \n {modelos_dict_RF1[familia].best_params_}')
    y_pred_dict_RF1[familia] = pd.DataFrame(modelos_dict_RF1[familia].predict(X_train_dict[familia]), index=y_train_dict[familia].index, columns=y_train_dict[familia].columns)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Ajustado modelo de AUTOMOTIVE 
 {'bootstrap': True, 'criterion': 'mse', 'max_depth': 5, 'max_features': None, 'max_samples': 0.6, 'n_estimators': 250, 'oob_score': True}
Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Ajustado modelo de BABY CARE 
 {'bootstrap': True, 'criterion': 'mse', 'max_depth': 5, 'max_features': None, 'max_samples': 1, 'n_estimators': 180, 'oob_score': True}
Fitting 5 folds for each of 18 candidates, totalling 90 fits


...

Ajustado modelo de PRODUCE 
 {'bootstrap': True, 'criterion': 'mse', 'max_depth': 11, 'max_features': None, 'max_samples': 0.6, 'n_estimators': 250, 'oob_score': True}
Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Ajustado modelo de SCHOOL AND OFFICE SUPPLIES 
 {'bootstrap': True, 'criterion': 'mse', 'max_depth': 11, 'max_features': None, 'max_samples': 0.6, 'n_estimators': 180, 'oob_score': True}
Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Ajustado modelo de SEAFOOD 
 {'bootstrap': True, 'criterion': 'mse', 'max_depth': 11, 'max_features': None, 'max_samples': 0.6, 'n_estimators': 250, 'oob_score': True}


In [109]:
X1_test_dict = {}
resul_actual = {}

for familia in familias:
    X1_test_dict[familia] = residuos_test_dict[familia].loc[:,'onpromotion':'y_lead_promotions_3']
    X1_test_dict[familia] = X1_test_dict[familia].reset_index()
    resul_actual[familia] = pd.concat([y_train_dict[familia].reset_index(),X1_test_dict[familia].reset_index()[['date','store_nbr']]]).set_index(['store_nbr','date']).sort_index()

In [110]:
fechas = pd.date_range(start="2017-08-16", end = "2017-08-31").to_period(freq='D').tolist()
lags_resi_test={}
X2_iteracion = {}
X1_iteracion = {}
X_iteracion = {}
pred_iter = {}

for fecha in fechas:
    for familia in familias:
        X1_iteracion[familia] = X1_test_dict[familia][X1_test_dict[familia]['date']==fecha]
        X1_iteracion[familia] = X1_iteracion[familia].set_index(['store_nbr','date']).sort_index()
        if familia in imp_lags.keys():
            lags_resi_test[familia] = make_lags_residuos(resul_actual[familia].residuos, lags = imp_lags[familia])
            lags_resi_test[familia].reset_index(inplace=True)
            X2_iteracion[familia] = lags_resi_test[familia][lags_resi_test[familia]['date']==fecha]
            X2_iteracion[familia] = X2_iteracion[familia].set_index(['store_nbr','date']).sort_index()
            X_iteracion[familia] = X1_iteracion[familia].join(X2_iteracion[familia])
        else:
            X_iteracion[familia] = X1_iteracion[familia]
        pred_iter[familia] = pd.DataFrame(modelos_dict_RF1[familia].predict(X_iteracion[familia]), index=X_iteracion[familia].index, columns = y_train_dict[familia].columns)
        resul_actual[familia] = resul_actual[familia].reset_index()
        resul_actual[familia] = resul_actual[familia][resul_actual[familia]['date'] != fecha]
        resul_actual[familia] = resul_actual[familia].set_index(['store_nbr', 'date']).sort_index()
        resul_actual[familia] = pd.concat([resul_actual[familia],pred_iter[familia]]).sort_index()    

In [111]:
y_fore_dict = {}

for familia in familias:
    y_fore_dict[familia] = resul_actual[familia].reset_index()
    y_fore_dict[familia] = y_fore_dict[familia][y_fore_dict[familia]['date']>='2017-08-16'].set_index(['store_nbr','date']).sort_index()

In [112]:
residuos_ajustados_test = pd.DataFrame()
for familia in familias:
    y_fore_dict[familia]['family']=familia
    residuos_ajustados_test = pd.concat([residuos_ajustados_test, y_fore_dict[familia]])

In [116]:
residuos_ajustados_test = residuos_ajustados_test.reset_index().set_index(['store_nbr', 'family', 'date']).sort_index().rename(columns={0:'predicted_residuos'})
y_test_pred = pd.DataFrame()
y_test_pred['adjusted_predicted_sales'] = y_fore_LR['predicted_sales'] + residuos_ajustados_test['residuos']
y_test_pred.loc[y_test_pred['adjusted_predicted_sales']<0,'adjusted_predicted_sales'] = 0
y_test_pred

adjusted_predicted_sales
store_nbr family     date                                
1         AUTOMOTIVE 2017-08-16                  4.178495
                     2017-08-17                  4.031175
                     2017-08-18                  5.298329
                     2017-08-19                  4.241971
                     2017-08-20                  1.647263
...                                                   ...
54        SEAFOOD    2017-08-27                  3.273146
                     2017-08-28                  3.140536
                     2017-08-29                  3.748062
                     2017-08-30                  4.625507
                     2017-08-31                  3.457665

[28512 rows x 1 columns]

In [117]:
y_submit = y_test_pred.join(store_sales_test.id)
y_submit.columns = ['sales','id']
y_submit = y_submit.set_index('id')
y_submit.sort_index(inplace=True)
y_submit

,sales
id,
3000888,4.178495
3000889,0.052566
3000890,7.356766
3000891,2822.000134
3000892,0.474689
...,...
3029395,372.525930
3029396,110.487406
3029397,1278.303445


In [118]:
y_submit.to_csv('submission9.csv')

**Empleando el Random Forest el resultado es mucho peor, 0.67587 de RMSLE**